tf-idf to find most frequent word for each sector/time period and theme

In [195]:
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np
import ast
from collections import Counter

In [196]:
arcom_df = pd.read_csv("data/ARCOM.csv")
arcom_df = arcom_df.dropna(subset=['clean_script'])
arcom_df.reset_index(drop=True, inplace=True)

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_4145/1574325814.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("data/ARCOM.csv")


In [197]:
arcom_df.columns

Index(['N° Fiche', 'year', 'week', 'N° Version', 'Annonceur', 'Marque',
       'Produit', 'Date', 'Date VP', 'Média', 'Medium', 'Catégorie', 'Support',
       'Format', 'Secteur', 'Classe', 'Groupe', 'Variété', 'N° Groupe/Variété',
       'Signature', 'Script', 'Incrustation', 'Titre', 'Libellé Version',
       'Visuel', 'Mots Clés', 'Thème', 'Musique / Auteur compositeur',
       'Musique / Titre', 'Agence', 'Réalisateur', 'Producteur', 'Récompense',
       'Musique / Version', 'Type de campagne', 'Musique / Interprète',
       'Partenaires', 'Titre émission', 'Hashtag', 'clean_script',
       'mots_clefs_themes', 'mots_clefs_themes_str', 'visuel_str',
       'visuel_clean', 'visuel_clean_bigram', 'clean_bigram_total', 'date_iso',
       'month_nb'],
      dtype='object')

In [198]:
for i, v in pd.Series(
        np.concatenate(
            arcom_df['mots_clefs_themes'].map(lambda x: [x.strip() for x in x.split(',')])
        )
    ).value_counts().items():
    print(i + ' : ' + str(v))
    if v <= 100: break

PRATIQUE : 10003
INTERPELLATION : 9854
LIEN MEDIA : 8661
MEDIA INTERNET : 8251
TYPOGRAPHIE : 7534
HUMOUR AUTRE : 6492
EFFICACITE : 6197
CHOIX : 5833
GOURMANDISE : 5660
ILLUSTRATION VISUELLE : 5479
NOUVEAUTE AUTRE : 5475
JOIE : 5258
SIGNATURE SONORE : 5195
BONNE HUMEUR : 5181
PERSONNE CELEBRE AUTRE : 5116
EFFET VISUEL : 5115
ENFANT : 5108
DISCOURS PRIX : 4955
INTERROGATION : 4943
ANGLAIS : 4597
GOUT : 4519
UNIVERS PROFESSIONNEL : 4421
SERVICE AUTRE : 4304
COMPORTEMENT AUTRE : 4099
COUPLE : 3917
MUSIQUE CELEBRE : 3852
COULEUR : 3849
ANIMAL AUTRE : 3821
CONSEIL : 3805
FEMME : 3778
QUOTIDIEN : 3760
FAMILLE : 3644
PARTENARIAT AUTRE : 3495
SIGNE DE QUALITE AUTRE : 3376
MUSIQUE AUTRE : 3357
ETONNEMENT : 3312
INNOVATION : 3265
COMPLICITE : 3250
VALORISATION : 3195
SIGNATURE ETRANGERE : 3077
TAG : 3068
AMITIE : 3063
UNIVERS URBAIN : 3019
SENSATIONS AUTRE : 2997
BIEN-ETRE : 2921
SAVOIR-FAIRE AUTRE : 2825
JEUNE : 2824
ENVIRONNEMENT AUTRE : 2820
SANTE : 2788
COMPOSANT SPECIAL : 2784
EFFET : 2731
A

In [199]:


x = arcom_df[['Script', 'clean_script']].sample(50)

# tf idf 

In [200]:
text_series = arcom_df['clean_script'].apply(lambda x: ast.literal_eval(x))
labels_series = arcom_df['mots_clefs_themes'].apply(lambda x: [y.strip() for y in x.split(',')] )

print(f'text count: {len(text_series)}')
print(f'label count: {len(labels_series)}')

text count: 47585
label count: 47585


In [201]:
dictionary = Dictionary(text_series)
dictionary
corpus = [dictionary.doc2bow(text) for text in text_series]
model = TfidfModel(corpus)

In [202]:
target_label = 'DEVELOPPEMENT DURABLE'

target_texts = text_series[[target_label in labels for labels in labels_series.values]]
target_corpus = [dictionary.doc2bow(text) for text in target_texts]
target_tfidf = model[target_corpus]



In [203]:
threshold = 0.5
word_dict = {}
for sentence in target_tfidf:
    for word in sentence:
        if word[1] > threshold:
            if dictionary[word[0]] not in word_dict:
                word_dict[dictionary[word[0]]] = 1
            else:
                word_dict[dictionary[word[0]]] += 1

In [204]:
dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))

{'cristaline': 35,
 'kia': 19,
 'lu': 15,
 'vahine': 15,
 'senseo': 15,
 'bosch': 14,
 'clio': 13,
 'sofinco': 13,
 'hyundai': 11,
 'rozana': 10,
 'hybride': 10,
 'schweppes': 10,
 'art': 9,
 'toyota': 9,
 'yorre': 9,
 'préférée': 9,
 'brita': 9,
 'tucson': 9,
 'contrex': 9,
 'rainett': 8,
 'total': 8,
 'connetable': 8,
 'juice': 8,
 'vittel': 8,
 'saumon': 8,
 'ikea': 7,
 'gamm': 7,
 'chouette': 7,
 'biocoop': 7,
 'olia': 7,
 'fiat': 7,
 '500': 7,
 'e85': 7,
 'bmw': 6,
 'nissan': 6,
 'aviva': 6,
 'like': 6,
 'citroen': 6,
 'ford': 6,
 'daunat': 6,
 'dacia': 6,
 'ev6': 6,
 'delice': 6,
 'expliquer': 6,
 'secondes': 6,
 'kona': 6,
 'mont': 5,
 'choisissons': 5,
 'vichy': 5,
 'bulles': 5,
 'sodastream': 5,
 'destop': 5,
 'téléphones': 5,
 '208': 5,
 'madrange': 5,
 'delichoc': 5,
 'kinder': 5,
 'knorr': 4,
 'tri': 4,
 'thonon': 4,
 'meeeuuuh': 4,
 'allez': 4,
 'bonneterre': 4,
 'celestins': 4,
 'leclerc': 4,
 'cheveux': 4,
 'cadum': 4,
 'avocat': 4,
 'maif': 4,
 'blanc': 4,
 'agriculture